# Creating Custom Agents

This notebook demonstrates how to create custom Llama Stack agents with tools.

## Prerequisites

- Llama Stack running at `http://localhost:5001`
- MyloWare installed: `pip install -e .`


In [ ]:
import sys
sys.path.insert(0, '../src')

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent

# Connect to Llama Stack
client = LlamaStackClient(base_url="http://localhost:5001")


## 1. Create a Simple Agent

Create an agent with basic instructions:


In [ ]:
agent = Agent(
    client=client,
    model="meta-llama/Llama-3.2-3B-Instruct",
    instructions="You are a helpful assistant that specializes in video production.",
    tools=[],
)

# Create a session
session_id = agent.create_session("demo-session")
print(f"Session created: {session_id}")


## 2. Using MyloWare Agent Factory

Use the config-driven factory for project-specific agents:


In [ ]:
from agents.factory import create_agent

# Create an ideator agent from config
# This loads config from data/shared/agents/ideator.yaml 
# merged with data/projects/motivational/agents/ideator.yaml
ideator = create_agent(
    client=client,
    project="motivational",
    role="ideator",
    vector_db_id="project_kb_motivational",
)

print("Created ideator agent")


## 3. Custom Tool Example

Create a custom tool using MyloWare's base class:


In [ ]:
from typing import Dict
from llama_stack_client.types import ToolParamDefinition
from tools.base import MylowareBaseTool

class VideoIdeaTool(MylowareBaseTool):
    """Generate video ideas based on a topic."""
    
    def get_name(self) -> str:
        return "generate_video_idea"
    
    def get_description(self) -> str:
        return "Generate creative video ideas for a given topic"
    
    def get_params_definition(self) -> Dict[str, ToolParamDefinition]:
        return {
            "topic": ToolParamDefinition(
                param_type="str",
                description="Topic for the video",
                required=True,
            ),
            "style": ToolParamDefinition(
                param_type="str",
                description="Video style (e.g., 'calm', 'energetic')",
                required=False,
                default="calm",
            ),
        }
    
    def run_impl(self, topic: str, style: str = "calm") -> dict:
        # In real implementation, this might call an API or use AI
        return {
            "idea": f"A {style} video about {topic}",
            "scenes": [
                f"Opening: Beautiful {topic} landscape",
                f"Middle: Close-up details of {topic}",
                f"End: Peaceful {topic} at sunset",
            ]
        }

# Use the custom tool
tool = VideoIdeaTool()
result = tool.run_impl(topic="mountains", style="serene")
print(result)
